In [ ]:
import requests
import csv
import pandas as pd
from datetime import datetime
import RPi.GPIO as GPIO
import smbus
import threading
import time

def upload2():
    try:
        line = ""
        f = open("sensor2.csv", 'r')
        for tmp in f:
            line += tmp
        f.close()
        data = {'data': line}
        myUrl = 'http://wiffy.io/sen/upload2.php'
        requests.post(myUrl, data=data).text
    except:
        print('ERROR')
        
def download2():
    try:
        myUrl = 'http://wiffy.io/sen/sensor2.csv'
        res = requests.post(myUrl).text
        print(res)
        sres = res.split("\n")
        f = open("sensor2.csv", 'w')
        for tmp in sres:
            f.write(tmp + "\n")
        f.close()
    except:
        print('ERROR')
        
I2C_CH = 1
BH1750_DEV_ADDR = 0x23

CONT_H_RES_MODE     = 0x10
CONT_H_RES_MODE2    = 0x11
CONT_L_RES_MODE     = 0x13
ONETIME_H_RES_MODE  = 0x20
ONETIME_H_RES_MODE2 = 0x21
ONETIME_L_RES_MODE  = 0x23

def readIlluminance():
    i2c = smbus.SMBus(I2C_CH)
    luxBytes = i2c.read_i2c_block_data(BH1750_DEV_ADDR, CONT_H_RES_MODE, 2)
    lux = int.from_bytes(luxBytes, byteorder='big')
    i2c.close()
    return lux

'''
 120초에 한번씩 돌면서 조도값 데이터에 업로드
'''
def readIlluminanceThread():
    while True:
        print('{0} lux'.format(readIlluminance()))
        
        download1()
        sensor2 = pd.read_csv("sensor2.csv")
        
        now = datetime.now()
        list = [now, readIlluminance()]
        
        sensor2.loc[len(df)] = [n for n in list]
        sensor2.to_csv("sensor2.csv", index = False) # 데이터 인풋
        upload2()
        time.sleep(30)
        
'''
 데이터를 계속해서 추가해주는 쓰레드
'''
thd = threading.Thread(target=readIlluminanceThread)
# 쓰레드를 데몬으로 설정
thd.daemon = True
# 쓰레드 시작
thd.start()